In [1]:
import rclpy as rp
from geometry_msgs.msg import Twist

In [2]:
rp.init()
test_node = rp.create_node('sample_test')

In [3]:
msg = Twist()
print(msg)

geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0))


In [19]:
msg.linear.x = 2.0
print(msg)
pub = test_node.create_publisher(Twist, '/base_controller/cmd_vel_unstamped', 10)
pub.publish(msg)

geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=2.0, y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=2.0))


In [20]:
import pyaudio
import wave
import whisper
import torch
from torch.cuda.amp import autocast

# Constants
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 3 # Adjust the duration of the recording as needed
FILENAME = "output.wav"

# Initialize PyAudio
pa = pyaudio.PyAudio()

# Load the whisper model
model = whisper.load_model("base")
tmp_list=[]

while True:
    # Open the stream
    stream = pa.open(format=FORMAT,
                     channels=CHANNELS,
                     rate=RATE,
                     input=True,
                     frames_per_buffer=CHUNK)

    print("Recording started...")

    frames = []
    tmp_list=[]

    # Record audio data
    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    # Stop the stream
    stream.stop_stream()
    stream.close()

    # Save the recorded data to a WAV file
    with wave.open(FILENAME, "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(pa.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b"".join(frames))

    print("Recording saved to", FILENAME)

    # Load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(FILENAME)
    audio = whisper.pad_or_trim(audio)

    # Make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # Detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # Convert mel spectrogram to Float data type
    mel = mel.float()

    # Decode the audio
    options = whisper.DecodingOptions()

    # Enable automatic mixed precision
    with autocast():
        result = whisper.decode(model, mel, options)

        tmp_list = result.text.split(' ')
        for word in tmp_list:
            print(word)
            word = word.lower()
            # if word[-1] == '.':
            #     word.replace('.','')

            if word == 'forward.':
                print(word)
                msg.linear.x = 5.0
                msg.angular.z = 0.0
                #print(msg)
                #pub = test_node.create_publisher(Twist, '/turtle1/cmd_vel', 10)
                pub = test_node.create_publisher(Twist, '/base_controller/cmd_vel_unstamped', 10)
                pub.publish(msg)
            elif word == 'back.':
                print(word)
                msg.linear.x = -5.0
                msg.angular.z = 0.0
                #print(msg)
                #pub = test_node.create_publisher(Twist, '/turtle1/cmd_vel', 10)
                pub = test_node.create_publisher(Twist, '/base_controller/cmd_vel_unstamped', 10)
                pub.publish(msg)
            elif word =='left.':
                msg.linear.x = 5.0
                msg.angular.z = 2.0
                #print(msg)
                #pub = test_node.create_publisher(Twist, '/turtle1/cmd_vel', 10)
                pub = test_node.create_publisher(Twist, '/base_controller/cmd_vel_unstamped', 10)
                pub.publish(msg)
            elif word =='right.':
                msg.linear.x = 5.0
                msg.angular.z =-2.0
                #print(msg)
                #pub = test_node.create_publisher(Twist, '/turtle1/cmd_vel', 10)
                pub = test_node.create_publisher(Twist, '/base_controller/cmd_vel_unstamped', 10)
                pub.publish(msg)


    # Print the recognized text
    #print(result.text)

    # Optionally, you can perform additional actions based on the recognized text
    # ...

    # Clear the recorded frames for the next iteration
    frames = []


Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
Okay.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
forward.
forward.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: ko
됐다.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
That's
it.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
Turn
left.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
Go
back.
back.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: ko
아,
고생하셨습니다.
Recording started...
Recording finished.
Recording saved to output.wav
Detected language: en
Go
back.
back.
Recording started...
Recording finished.
Recording saved to output.wav


KeyboardInterrupt: 